# Minimal OpenFOAM Example with DAPI
## Running OpenFOAM simulations using DesignSafe API (DAPI)

## Install DesignSafe API (dapi)

In [ ]:
# Dapi installation
!pip uninstall dapi --yes

!pip install dapi --user --quiet

# Install the latest development version of dapi from GitHub
# !pip install git+https://github.com/DesignSafe-CI/dapi.git@dev --user --quiet

# Install editable local version of dapi
# !pip install -e ../

In [ ]:
# Import required modules
from dapi import DSClient
import json

In [ ]:
# Initialize DesignSafe client
ds = DSClient()

In [ ]:
# Job configuration parameters
ds_path: str = "/MyData/template-notebooks/tapis3/OpenFOAM/DH1_run"  # Path to OpenFOAM case directory
max_job_minutes: int = 10  # Maximum runtime in minutes
tacc_allocation: str = "ASC25049"  # TACC allocation to charge
app_id_to_use = "openfoam-stampede3"  # OpenFOAM application ID

# OpenFOAM-specific environment variables
openfoam_env_vars = [
    {"key": "mesh", "value": "On"},  # Enable mesh generation
    {"key": "solver", "value": "pisoFoam"},  # CFD solver to use
    {"key": "decomp", "value": "On"},  # Enable domain decomposition for parallel runs
]

In [ ]:
# Convert DesignSafe path to Tapis URI format
input_uri = ds.files.translate_path_to_uri(ds_path)
print(f"Input Directory Tapis URI: {input_uri}")

In [ ]:
# Generate job request dictionary using app defaults
job_dict = ds.jobs.generate_request(
    app_id=app_id_to_use,
    input_dir_uri=input_uri,
    max_minutes=max_job_minutes,
    allocation=tacc_allocation,
    archive_system="designsafe",
    extra_env_vars=openfoam_env_vars,
    input_dir_param_name="Case Directory",  # OpenFOAM apps use "Case Directory" instead of "Input Directory"
)
print(json.dumps(job_dict, indent=2, default=str))

In [ ]:
# Customize job settings (optional)
job_dict["nodeCount"] = 1  # Use single node
job_dict["coresPerNode"] = 2  # Use 2 cores for parallel simulation
print(json.dumps(job_dict, indent=2, default=str))

In [ ]:
# Submit the job to TACC
submitted_job = ds.jobs.submit_request(job_dict)
print(f"Job UUID: {submitted_job.uuid}")

In [ ]:
# Monitor job execution until completion
final_status = submitted_job.monitor(interval=15)  # Check every 15 seconds
print(f"Job {submitted_job.uuid} finished with status: {final_status}")

In [ ]:
# Display job runtime summary
submitted_job.print_runtime_summary(verbose=False)

In [ ]:
# Get current job status
current_status = ds.jobs.get_status(submitted_job.uuid)
print(f"Current status: {current_status}")

In [ ]:
# Display last status message from TACC
print(f"Last message: {submitted_job.last_message}")

In [ ]:
# Display job output from stdout
stdout_content = submitted_job.get_output_content("tapisjob.out", max_lines=50)
if stdout_content:
    print("Job output:")
    print(stdout_content)

In [ ]:
# List contents of job archive directory
archive_uri = submitted_job.archive_uri
print(f"Archive URI: {archive_uri}")
outputs = ds.files.list(archive_uri)
for item in outputs:
    print(f"- {item.name} ({item.type})")

## Post-processing (Optional)
### Visualize results with ParaView or extract force coefficients

The simulation results can be visualized using ParaView by opening the `foam.foam` file in the case directory.

For force coefficient analysis, check the `postProcessing/forceCoeffs1/0/` directory in the job archive.

## Post-processing\n### Extract and plot force coefficients from simulation results\n\nThe simulation results can be visualized using ParaView by opening the `foam.foam` file in the case directory.\n\nFor force coefficient analysis, check the `postProcessing/forceCoeffs1/0/` directory in the job archive."

In [ ]:
# Get DesignSafe Jupyter path
archive_path = ds.files.translate_uri_to_path(archive_uri)
print(archive_path)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os

In [ ]:
# Load force coefficients data using pandas for cleaner parsing
import pandas as pd

force_data_path = (
    archive_path + "/inputDirectory/postProcessing/forceCoeffs1/0/forceCoeffs.dat"
)

# Read the file, skipping header lines and using tab separator
data = pd.read_csv(force_data_path, sep="\t", skiprows=9, header=None)
print(f"Loaded force coefficients data with shape: {data.shape}")

In [ ]:
# Plot drag coefficient (Cd) vs time
plt.plot(data.iloc[100:, 0], data.iloc[100:, 2])
plt.xlabel("Time")
plt.ylabel("$C_d$")
plt.title("Drag Coefficient vs Time")
plt.grid(False)
plt.show()

In [ ]:
# Plot lift coefficient (Cl) vs time
plt.plot(data.iloc[100:, 0], data.iloc[100:, 3])
plt.xlabel("Time")
plt.ylabel("$C_l$")
plt.title("Lift Coefficient vs Time")
plt.grid(False)
plt.show()